In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [17]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(ChromeDriverManager().install())
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    driver.maximize_window()
    
    
    # =====================================================================================
    # get the login icon and click on it 
# =====================================================================================
    # try:
    #         # get the login button
    #         login = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Login"]')))
    #         if login:
    #             driver.execute_script("arguments[0].scrollIntoView();", login)
    #             login.click()
    #         else:
    #             pass
    # except Exception as e:
    #     print(f"No login tab found and the error is {e}") 
    
    # #  get the username tab
    # try:
    #     provided_u_name = "fourbrotherstrading@icloud.com"
    #     user_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID,"logonIdentifier")))   
    #     user_name.send_keys(provided_u_name)
    # except Exception as e:
    #     print(f"No username tab found and the error is {e}")
    
    # # get password tab
    # try:
    #     provided_pass = "Muhssan7865"
    #     password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password")))
    #     driver.execute_script("arguments[0].scrollIntoView();", password)
    #     password.send_keys(provided_pass)
    # except Exception as e:
    #     print(f"No password tab found and the error is {e}") 
    
    # try:
    #     sign =  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/button[text()= "Sign in"]')))
    #     if sign:
    #         sign.click()
    # except Exception as e:
    #     print(f"No sign in found {e}")
    
    
    
    
    # total cars
    try:
        cars=  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//p[@class="MuiTypography-root MuiTablePagination-caption MuiTypography-body2 MuiTypography-colorInherit"])[2]')))
        if cars:
            tot_cars= int(cars.text.strip().split(" ")[2])
            print(f"found {tot_cars} cars ")
    except Exception as e:
        print(f"No cars found {e}")
    
    # get the check done
    try:
        check = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/button[@id="rcc-confirm-button"]')))
        if check:
            driver.execute_script("arguments[0].scrollIntoView();", check)
            check.click()
        
    except Exception as e:
        print(f"No check tab found and the error is {e}")
    
    # try:
    #     page_cars = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-2 MuiGrid-item"]')))
    #     if page_cars:
    #         print("found page cars")
    #         print(len(page_cars))
    # except Exception as e:
    #     print(e)
    
    
    # result
    results = []
    car_count = 0
    while car_count < tot_cars: #total_cars
        
        try:
            page_cars = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-2 MuiGrid-item"]')))
            
            for i in range(len(page_cars)): #total_cars
                if car_count >= tot_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-2 MuiGrid-item"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(3)
                    page_cars[i].click()

                    details = {}
                    
                    # car title and category
                    try:
                        tit_main =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-1"])[1]'))) 
                        if tit_main:
                            cate = WebDriverWait(tit_main, 5).until(EC.presence_of_element_located((By.XPATH, '(.//span[@class="MuiChip-label MuiChip-labelSmall"])[1]'))).text.strip()
                            if cate:
                                details['Category'] = cate.split(" ")[1]
                            else:
                                details['Category'] = "na"
                                
                            title =WebDriverWait(tit_main, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'h2'))).text.strip()
                            if title:
                                details['Title'] = title
                            else:
                                details['Title'] = "na"
                    except Exception as e:
                        print(f"no car name found {e}")
                    
                    # date and time
                    try:
                        date_time = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-2"]'))).text.strip()
                        if date_time:
                            details['Date_time'] = date_time
                        else:
                            details['Date_time'] = "na"
                    except Exception as e:
                        print(f"No date andm time {e}")
                    
                    
                    # images --> done
                    try:
                        imgs_main = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="slick-track"]')))
                        if imgs_main:
                            # print("images slide found")
                            imgs=WebDriverWait(imgs_main,5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[contains(@class, "MuiCardMedia-root")]'))) 
                            if imgs:
                                # print("images found ")
                                imgs_lst = [img.get_attribute("style").split(": ")[1].split('("')[1].split('")')[0] for img in imgs]
                                details['Images'] = ", ".join(imgs_lst)
                    except Exception as e:
                        print(f"no images {e}")
                        
                        
                    # other information
                    try:
                        tabs_main = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="MuiTabs-flexContainer"]')))
                        if tabs_main:
                            
                            # lot details 
                            try:
                                tab_lot_seller = WebDriverWait(tabs_main, 5).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Lot Details"]')))
                                if tab_lot_seller:
                                    driver.execute_script("arguments[0].scrollIntoView();", tab_lot_seller)
                                    # time.sleep(2)
                                    tab_lot_seller.click()
                                    
                                    lot_main = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-1"])[last()]')))
                                    if lot_main:
                                        lot_left_info =WebDriverWait(lot_main, 5).until(EC.presence_of_element_located((By.XPATH, '(.//ul[@class="MuiList-root MuiList-padding"])[1]'))) 
                                        if lot_left_info: 
                                            lot_vals = WebDriverWait(lot_left_info, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/span[@class="MuiTypography-root MuiListItemText-primary MuiTypography-body1 MuiTypography-displayBlock"]')))
                                            if lot_vals:
                                                # print("loc vals found")
                                                lots = [lot_val.text.strip() for lot_val in lot_vals]
                                                details['lot_dets'] = lots
                                    else:
                                        print("No lot details")
                            except Exception as e:
                                print("No lot details found")
                            
                            
                            # seller details
                            try:
                                tab_lot_seller = WebDriverWait(tabs_main, 5).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Seller Details"]')))
                                if tab_lot_seller:
                                    driver.execute_script("arguments[0].scrollIntoView();", tab_lot_seller)
                                    # time.sleep(2)
                                    tab_lot_seller.click()
                                    
                                    sell_main_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-5"])[2]'))) 
                                    if sell_main_card:
                                        
                                        # location --> done
                                        location_card = WebDriverWait(sell_main_card, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-md-6"])[1]')))
                                        if location_card:
                                            # print("Location main card found")
                                            loc_vals = WebDriverWait(location_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/p[@class="MuiTypography-root MuiTypography-body1 MuiTypography-alignLeft"]')))
                                            if loc_vals:
                                                # print("loc vals found")
                                                locs = [loc_val.text.strip() for loc_val in loc_vals]
                                                details['Seller_Location'] = locs
                                    
                                        else:
                                            print("No location details")
                                        
                                        # openning times
                                        open_times_card = WebDriverWait(sell_main_card, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-md-6"])[3]')))
                                        if open_times_card:
                                            # print("Location main card found")
                                            open_vals = WebDriverWait(open_times_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/p[@class="MuiTypography-root MuiTypography-body1 MuiTypography-alignLeft"]')))
                                            if open_vals:
                                                # print("loc vals found")
                                                opens = [open_val.text.strip() for open_val in open_vals]
                                                details['Seller_Opening_times'] = opens
                                        else:
                                            print("No openning details")
                            except Exception as e:
                                print(f"No seller information {e}")  
                            
                            
                            # location details
                            try:
                                tab_lot_location = WebDriverWait(tabs_main, 5).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Location Details"]')))
                                if tab_lot_location:
                                    driver.execute_script("arguments[0].scrollIntoView();", tab_lot_location)
                                    # time.sleep(2)
                                    tab_lot_location.click()
                                    
                                    loc_main_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-3"])[1]'))) 
                                    if loc_main_card:    
                                        # location --> done
                                        loc_card = WebDriverWait(loc_main_card, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-md-6"])[1]')))
                                        if loc_card:
                                            # print("Location main card found")
                                            location_vals = WebDriverWait(loc_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/p[@class="MuiTypography-root MuiTypography-body1 MuiTypography-alignLeft"]')))
                                            if location_vals:
                                                # print("loc vals found")
                                                locations = [location_val.text.strip() for location_val in location_vals]
                                                details['Location'] = locations
                                    
                                        else:
                                            print("No location details")
                                        
                                        # openning times
                                        open_card = WebDriverWait(loc_main_card, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-md-6"])[3]')))
                                        if open_card:
                                            # print("Location main card found")
                                            open_dets = WebDriverWait(open_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/p[@class="MuiTypography-root MuiTypography-body1 MuiTypography-alignLeft"]')))
                                            if open_dets:
                                                # print("loc vals found")
                                                opens_dets = [open_det.text.strip() for open_det in open_dets]
                                                details['Opening_times'] = opens_dets
                                        else:
                                            print("No opening details")
                                            
                                        # collection times
                                        collection_card = WebDriverWait(loc_main_card, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-md-6"])[4]')))
                                        if collection_card:
                                            # print("Location main card found")
                                            collection_dets = WebDriverWait(collection_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/p[@class="MuiTypography-root MuiTypography-body1 MuiTypography-alignLeft"]')))
                                            if collection_dets:
                                                # print("loc vals found")
                                                collections_dets = [collection_det.text.strip() for collection_det in collection_dets]
                                                details['Collection_times'] = collections_dets
                                        else:
                                            print("No collection details")
                            except Exception as e:
                                print(f"No seller information {e}")  
                            
                            
                            
                    except Exception as e:
                        print("No other info found")         
                    
                    
                    car_count+=1
                    results.append(details)
                    driver.back()
                except Exception as e:
                    pass
                
            if car_count % 20 == 0: # --> 25%25 == 0 and 50%25== 0 --> issue here
                    try:
                        next_link = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//button[contains(@title, "Next page") and not (contains(@class, "MuiButtonBase-root MuiIconButton-root MuiIconButton-colorInherit Mui-disabled Mui-disabled"))])[2]')))
                        if next_link:
                            driver.execute_script("arguments[0].scrollIntoView();", next_link)
                            time.sleep(2)
                            next_link.click()
                            WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-2 MuiGrid-item"]')))
                            print("Hit the next button")
                    except Exception as e:
                        print("No next found")
                        break
        except Exception as e:
            print("No more car links")
            break
                
                           
    df = pd.DataFrame.from_dict(results)
    df.to_csv("ams.csv", index=False)
    time.sleep(5)
    driver.quit()
    
path = "https://auctions.asm-autos.co.uk/Search?auction[]=&bucketDetails=Pic%20Up%20Spares%20Ltd%20-%20Friday%2C%2021%20February%202025&bucketId=056d6bde-070c-40b0-bc23-1cb3b4c009c4&damageCategory[]=&distance[]=&editorPickSearch=0&fuelType[]=&latitude=0&longitude=0&make[]=&model[]=&orderBy=1&pageNumber=0&pageSize=20&quickSearch=0&searchText=&seller[]=&startDrive[]=&transmissionType[]=&year[]="
scrape(path)

found 33 cars 
Hit the next button


In [21]:
df = pd.read_csv("ams.csv")
df['Images'][1]

'https://stsmlotimagesprod.blob.core.windows.net/992f3627-6f91-4c94-bb8d-18e33037fe0e/debcd363-90b6-484a-ac9c-cbbe6faaa539.jpg, https://stsmlotimagesprod.blob.core.windows.net/992f3627-6f91-4c94-bb8d-18e33037fe0e/e46baa55-2581-453b-a1cb-5707e990c753.jpg, https://stsmlotimagesprod.blob.core.windows.net/992f3627-6f91-4c94-bb8d-18e33037fe0e/48b1506f-6190-44ce-aedf-1e215b0c5070.jpg, https://stsmlotimagesprod.blob.core.windows.net/992f3627-6f91-4c94-bb8d-18e33037fe0e/c8f2f355-7019-4f7e-9ee1-e930a1e58753.jpg, https://stsmlotimagesprod.blob.core.windows.net/992f3627-6f91-4c94-bb8d-18e33037fe0e/6e879ef2-b445-4282-836f-c31fa93556cc.jpg, https://stsmlotimagesprod.blob.core.windows.net/992f3627-6f91-4c94-bb8d-18e33037fe0e/43107a0e-10d7-4345-a9b4-832a1e1211b2.jpg, https://stsmlotimagesprod.blob.core.windows.net/992f3627-6f91-4c94-bb8d-18e33037fe0e/8f8a2c5f-519f-4ac9-bcc1-21faf455d838.jpg, https://stsmlotimagesprod.blob.core.windows.net/992f3627-6f91-4c94-bb8d-18e33037fe0e/c67e8a89-b8c2-4474-b22f-6

# image downloader

In [ ]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.
reg_img = df[['Title', 'Images']]


def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Title"] # separate reg nums
        image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # loop around the urls of the current row and also save index for further use
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                url = urljoin("https://", url) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(url)

            # check if the parsed url is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                continue
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(url, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Extracts the file extension (e.g., jpg). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")
# Call the function
download_images(reg_img)
